<a href="https://colab.research.google.com/github/BucketofJava/EnglishToPhonetics/blob/main/EnglishToPhonetics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#English to Phonetics

####Imports

In [1]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import urllib.request as URL
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.sequence as kps
from torch import nn;
from torch import Tensor;
from torch import optim;
from torch import reshape, argmax, zeros, randn, zeros_like, save;
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader;
from torchinfo import summary as tisummary;
from torch.utils.tensorboard import *
from torch import autograd
import torch.cuda
from random import *
import time
from datetime import datetime
from torch.nn.functional import one_hot

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

##Data Processing

####Fetching Data

In [4]:
dictionary_url="https://github.com/cmusphinx/cmudict/raw/master/"
dictionary_fileName="cmudict.dict"
URL.urlretrieve(dictionary_url+dictionary_fileName, dictionary_fileName)
dict_file="/content/"+dictionary_fileName

In [5]:
f=open(dict_file, "r")
full_text=f.read()
f.close()
full_text_split=full_text.split("\n")
word_list=[]
phonetics_list=[]
for i in range(len(full_text_split)):
  full_text_split[i]=full_text_split[i].split(" ")
  full_text_split[i].insert(1, "SOS")
  full_text_split[i].append("EOS")
  word_list.append(list(full_text_split[i][0]))
  phonetics_list.append(full_text_split[i][1:])
print(full_text_split[:1000])
print(phonetics_list[:1000])

[["'bout", 'SOS', 'B', 'AW1', 'T', 'EOS'], ["'cause", 'SOS', 'K', 'AH0', 'Z', 'EOS'], ["'course", 'SOS', 'K', 'AO1', 'R', 'S', 'EOS'], ["'cuse", 'SOS', 'K', 'Y', 'UW1', 'Z', 'EOS'], ["'em", 'SOS', 'AH0', 'M', 'EOS'], ["'frisco", 'SOS', 'F', 'R', 'IH1', 'S', 'K', 'OW0', 'EOS'], ["'gain", 'SOS', 'G', 'EH1', 'N', 'EOS'], ["'kay", 'SOS', 'K', 'EY1', 'EOS'], ["'m", 'SOS', 'AH0', 'M', 'EOS'], ["'n", 'SOS', 'AH0', 'N', 'EOS'], ["'round", 'SOS', 'R', 'AW1', 'N', 'D', 'EOS'], ["'s", 'SOS', 'EH1', 'S', 'EOS'], ["'til", 'SOS', 'T', 'IH1', 'L', 'EOS'], ["'tis", 'SOS', 'T', 'IH1', 'Z', 'EOS'], ["'twas", 'SOS', 'T', 'W', 'AH1', 'Z', 'EOS'], ['a', 'SOS', 'AH0', 'EOS'], ['a(2)', 'SOS', 'EY1', 'EOS'], ["a's", 'SOS', 'EY1', 'Z', 'EOS'], ['a.', 'SOS', 'EY1', 'EOS'], ["a.'s", 'SOS', 'EY1', 'Z', 'EOS'], ['a.d.', 'SOS', 'EY2', 'D', 'IY1', 'EOS'], ['a.m.', 'SOS', 'EY2', 'EH1', 'M', 'EOS'], ['a.s', 'SOS', 'EY1', 'Z', 'EOS'], ['aaa', 'SOS', 'T', 'R', 'IH2', 'P', 'AH0', 'L', 'EY1', 'EOS'], ['aaberg', 'SOS', 'AA

###Tokenize phonetics & letters

In [6]:
letter_tokenizer=Tokenizer()

flattened_wordlist=[character for word in word_list for character in word]
letter_tokenizer.fit_on_texts(flattened_wordlist)
tokenized_words=letter_tokenizer.texts_to_sequences(word_list)
tokenized_words=kps.data_utils.pad_sequences(tokenized_words, padding="post", maxlen=32)[:]

In [7]:
phonetic_tokenizer=Tokenizer()
flattened_phoneticlist=[phoneme for word in phonetics_list for phoneme in word]
phonetic_tokenizer.fit_on_texts(flattened_phoneticlist)
tokenized_phonetics=phonetic_tokenizer.texts_to_sequences(phonetics_list)
tokenized_phonetics=kps.data_utils.pad_sequences(tokenized_phonetics, padding="post", maxlen=24)[:]

In [8]:
start_token=phonetic_tokenizer.texts_to_sequences([['SOS']])[0][0]
end_token=phonetic_tokenizer.texts_to_sequences([['EOS']])[0][0]

In [9]:
word_index_inverted={v: k for k,v in phonetic_tokenizer.word_index.items()}

In [ ]:
print(phonetic_tokenizer.texts_to_sequences([['SOS']]))

In [ ]:
print(tokenized_phonetics)

In [10]:
phonetic_length=tokenized_phonetics.shape[1]
word_length=tokenized_words.shape[1]

In [11]:
tokenized_phonetics=tokenized_phonetics.astype('float32')
tokenized_words=tokenized_words.reshape(tokenized_words.shape[0], tokenized_words.shape[1]).astype('float32')


In [ ]:
print(tokenized_phonetics.shape)

(135156, 32)


In [ ]:
print(tokenized_words.shape)

In [12]:
indices=np.load("/content/drive/MyDrive/indices_2022-12-22 19:41:54.023788.npy")

In [12]:
ids_file=open("/content/indices_2022-12-21 23_39_03.650224.txt", "r")
result=ids_file.read()
result=result.replace("[", "")
result=result.split(" ")
print(result)
indices=[]
for i in result:
  if(i==""):
    continue;
indices=np.array(indices)
print(indices)

FileNotFoundError: ignored

In [18]:
indices=np.array(range(tokenized_words.shape[0]))
np.random.shuffle(indices)
tokenized_phonetics=tokenized_phonetics[indices]
tokenized_words=tokenized_words[indices]


In [13]:
tokenized_phonetics=tokenized_phonetics[indices]
tokenized_words=tokenized_words[indices]

In [20]:
np.save("/content/drive/MyDrive/indices_"+str(datetime.now()), indices)

In [ ]:
print(tokenized_phonetics)

[[ 1.  5. 17. ...  0.  0.  0.]
 [ 1.  4. 36. ...  0.  0.  0.]
 [ 1. 22.  8. ...  0.  0.  0.]
 ...
 [ 1.  6. 36. ...  0.  0.  0.]
 [ 1.  5.  6. ...  0.  0.  0.]
 [ 1.  8. 34. ...  0.  0.  0.]]


In [ ]:
tokenized_phonetics=one_hot(torch.tensor(tokenized_phonetics).long(), num_classes=len(phonetic_tokenizer.word_index)+1)

In [14]:
tokenized_phonetics=torch.tensor(tokenized_phonetics).long()

In [ ]:
print(tokenized_words.shape)

In [ ]:
print(tokenized_words[3000:3100])

In [ ]:
print(tokenized_words.shape)

(135156, 32)


In [ ]:
print(tokenized_phonetics.shape)

torch.Size([135156, 32, 84])


In [ ]:
print(tokenized_words.shape)

(135156, 32)


In [15]:
split_index=int(tokenized_words.shape[0]*0.8)

words_train_tensor, words_test_tensor=(torch.tensor(tokenized_words[:split_index]), torch.tensor(tokenized_words[split_index:]))
phonetics_train_tensor, phonetics_test_tensor=(tokenized_phonetics[:split_index], tokenized_phonetics[split_index:])


In [16]:
words_train_tensor=words_train_tensor[1980*32:]
phonetics_train_tensor=phonetics_train_tensor[1980*32:]

In [ ]:
print(phonetics_train_tensor[:6])

In [ ]:
print(words_train_tensor)

tensor([ 6.,  4., 15., 14.,  8.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])


In [17]:
etp_train_dataset=TensorDataset(words_train_tensor.long(), phonetics_train_tensor.float())
etp_train_dataloader=DataLoader(etp_train_dataset, batch_size=32, num_workers=2)
etp_test_dataset=TensorDataset(words_test_tensor.long(), phonetics_test_tensor.float())
etp_test_dataloader=DataLoader(etp_test_dataset, batch_size=32, num_workers=2)

## Model Definition and Training

In [18]:
class ETPEncoder(nn.Module):
  def __init__(self, num_word_chars, embedding_size, hidden_state_size, num_encoder_layers):
    super(ETPEncoder, self).__init__()
    #Defining Encoder Model: 
    #Input: Sequence of tokens representing letters of a given word
    #Output: Cell states for each letter and the final hidden state
    self.encoder=nn.Sequential(
        #Embedding layer: Tokens -> Vectors
        nn.Embedding(num_word_chars, embedding_size),
        #LSTM layer(s): Vectorized Tokens -> Hidden and Output states
        nn.LSTM(input_size=embedding_size, hidden_size=hidden_state_size, num_layers=num_encoder_layers, batch_first=True)
    );
  def forward(self, x):
    #Returns: LSTM output & Final Hidden State
    return self.encoder(x)[1]

In [19]:
class ETPEncoderGRU(nn.Module):
  def __init__(self, num_word_chars, embedding_size, hidden_state_size, num_encoder_layers):
    super(ETPEncoderGRU, self).__init__()
    #Defining Encoder Model: 
    #Input: Sequence of tokens representing letters of a given word
    #Output: Cell states for each letter and the final hidden state
    self.encoder=nn.Sequential(
        #Embedding layer: Tokens -> Vectors
        nn.Embedding(num_word_chars, embedding_size),
        #LSTM layer(s): Vectorized Tokens -> Hidden and Output states
        nn.GRU(input_size=embedding_size, hidden_size=hidden_state_size, num_layers=num_encoder_layers, batch_first=True)
    );
  def forward(self, x):
    #Returns: LSTM output & Final Hidden State
    return self.encoder(x)[1]

In [20]:
class ETPDecoderGRU(nn.Module):
  def __init__(self, num_phonetic_chars, embedding_size, hidden_state_size, num_decoder_layers, desired_output_length=32, dropout=1):
    super(ETPDecoderGRU,self).__init__()
    #Embedding: Tokens representing phonemes -> Vectors, Output Size=embedding_size
    self.decoder_embedding=nn.Embedding(num_phonetic_chars, embedding_size)
    #LSTM: Sequence of Embedding Vectors -> Output Value, Output Size=hidden_state_size
    self.decoder_GRU=nn.GRU(input_size=embedding_size, hidden_size=hidden_state_size, num_layers=num_decoder_layers, batch_first=True, dropout=0.2)
    #Feedforward: LSTM Output -> Weight for each phoneme for a given character, Output Size=num_phonetic_chars
    self.decoder_feedforward= nn.Linear(hidden_state_size, num_phonetic_chars)
    #Log Softmax: Feedforward output -> Softmax (Percent of each) -> Natural Logarithm
    self.decoder_activation=   nn.LogSoftmax(dim=1)
    self.desired_output_length=desired_output_length
  def forward(self, x, GRU_hidden, batch_first=True):
    # print(GRU_hidden.size())
    # if(batch_first):
    #   GRU_hidden=GRU_hidden.transpose(0, 1)[-1:]
    # print("ASD")
    # print(x)
    embedded_input=self.decoder_embedding(x);
    GRU_result=self.decoder_GRU(embedded_input, GRU_hidden)
    # print("Drifesd")
    # print(GRU_result[0].size())
    # print("Feedforward result")
    feedforward_output=self.decoder_feedforward(GRU_result[0])
    # print(feedforward_output.size())
    activation_output=self.decoder_activation(feedforward_output)
    # print("Acitivation:")
    # print(activation_output.size())
    #Returns: Result of decoder given input & final hidden state
    return (activation_output, GRU_result[1])

In [21]:
class ETPDecoder(nn.Module):
  def __init__(self, num_phonetic_chars, embedding_size, hidden_state_size, num_decoder_layers, desired_output_length=32):
    super(ETPDecoder, self).__init__()
    #Embedding: Tokens representing phonemes -> Vectors, Output Size=embedding_size
    self.decoder_embedding=nn.Embedding(num_phonetic_chars, embedding_size)
    #LSTM: Sequence of Embedding Vectors -> Output Value, Output Size=hidden_state_size
    self.decoder_lstm=nn.LSTM(input_size=embedding_size, hidden_size=hidden_state_size, num_layers=num_decoder_layers, batch_first=True)
    #Feedforward: LSTM Output -> Weight for each phoneme for a given character, Output Size=num_phonetic_chars
    self.decoder_feedforward= nn.Linear(hidden_state_size, num_phonetic_chars)
    #Log Softmax: Feedforward output -> Softmax (Percent of each) -> Natural Logarithm
    self.decoder_activation=   nn.LogSoftmax(dim=1)
    self.desired_output_length=desired_output_length
  def forward(self, x, lstm_hidden, batch_first=True):
    if(batch_first):
      lstm_hidden=(lstm_hidden[0].transpose(0, 1)[-1:], lstm_hidden[1].transpose(0, 1)[-1:])
    lstm_result=self.decoder_lstm(self.decoder_embedding(x), lstm_hidden)
    print("Drifesd")
    print(lstm_result[0].size())
    #Returns: Result of decoder given input & final hidden state
    return (self.decoder_activation(self.decoder_feedforward(lstm_result[0])), lstm_result[1])

In [22]:
autograd.set_detect_anomaly(True)

In [23]:
class ETPModelV2(nn.Module):
  def __init__(self, num_word_chars, num_phonetic_chars, embedding_size, hidden_state_size, num_encoder_layers, num_decoder_layers, desired_output_length=32, use_gru=False):
    super(ETPModelV2, self).__init__();
    self.use_gru=use_gru
    self.encoder_model=ETPEncoder(num_word_chars+1, embedding_size, hidden_state_size, num_encoder_layers)
    self.decoder_model=ETPDecoder(num_phonetic_chars+1, hidden_state_size, num_decoder_layers, desired_output_length)
    if(use_gru):
        self.encoder_model=ETPEncoderGRU(num_word_chars+1, embedding_size, hidden_state_size, num_encoder_layers)
        self.decoder_model=ETPDecoderGRU(num_phonetic_chars+1,embedding_size, hidden_state_size, num_decoder_layers, desired_output_length)
    self.num_word_chars=num_word_chars+1;
    self.num_phonetic_chars=num_phonetic_chars+1;
    self.desired_output_length=desired_output_length;
  def forward(self, x, y=None, teacher_forcing_percent=0):
    #N=batch_size, L=self.desired_output_length, O=self.num_phonetic_chars, H=hidden_state_size
    batch_size=x.size()[0]
    #LxNxH -> 1xNxH
    current_hidden=self.encoder_model(x);
    #NxLxO
    predictions=zeros(batch_size, self.desired_output_length, self.num_phonetic_chars)
    predictions[:, 0, 1]=1;
    #NxL
    sequences=zeros(batch_size, self.desired_output_length)
    sequences[:, 0]=1;
    sequences=sequences.long()
    if(random()>teacher_forcing_percent):
      #Runs L-2 times
      for i in range(1, self.desired_output_length):
          #Inputs: NxL and 1xNxH
          #Outputs: Tuple: (NxLxO, LxNxH)
          if(y!=None and y[:, i-1].sum()==0):
             break;
          decoder_result=self.decoder_model(sequences.clone(), current_hidden)
          predictions[:, i]=decoder_result[0][:, -1]
          sequences[:, i]=decoder_result[0][:, -1].cpu().argmax(dim=1)
          #Shape: (1xNxH)
          current_hidden=decoder_result[1]
    else:
      for i in range(1, self.desired_output_length):
          #Inputs: NxL and 1xNxH
          #Outputs: Tuple: (NxLxO, LxNxH)
          if(y!=None and y[:, i-1].sum()==0):
             break;
          prev_sequence=zeros(batch_size, self.desired_output_length)
          prev_sequence[:, :i]=y[:, :i].clone()
          decoder_result=self.decoder_model(prev_sequence.long(), current_hidden)
          predictions[:, i]=decoder_result[0][:, -1]
          sequences[:, i]=decoder_result[0][:, -1].cpu().argmax(dim=1)
          #Shape: (1xNxH)
          current_hidden=decoder_result[1]
    return (predictions, sequences)
  def train_model(self, dataloader, loss_func, encoder_optimizer, decoder_optimizer):
    self.train();
    c=0;
    for batch, (x, y) in enumerate(dataloader):
      #Inputs: NxL
      #Outputs: Tuple: (NxLxO, NxL)
      (predictions, sequences)= self(x, y, teacher_forcing_percent=0.5);
      print(predictions.size())
      loss=0;
      for i in range(predictions.size()[1]):
        loss+=loss_func(predictions[i], y[i].long())
      if(c%5==0):
        print(f'Loss for data point {c*32}: {loss}')
      if(c%30==0):
        torch.save(self, "/content/drive/MyDrive/ETPmodel_{}_{}.pt".format(c, str(datetime.now())))   
        with torch.no_grad():
          result=self(torch.tensor(letter_tokenizer.texts_to_sequences("Spraul")).long())[1][0]
          s=""
          for i in result:
            s=s+","
            if(i==0):
              s=s+"0"
              continue
            s=s+word_index_inverted[i.item()]
          print(s)

      encoder_optimizer.zero_grad()
      decoder_optimizer.zero_grad()
      loss.backward()
      encoder_optimizer.step()
      decoder_optimizer.step()
      c+=1


###ETP Model V1

In [30]:
class ETPModel(nn.Module):
  def __init__(self, num_word_chars, num_phonetic_chars, embedding_size, hidden_state_size, num_encoder_layers, num_decoder_layers, desired_output_length=32, use_gru=False):
    super(ETPModel, self).__init__()
    #Defines Encoder Model
    self.use_gru=use_gru;
    if(not use_gru):
      self.encoder_model=ETPEncoder(num_word_chars, embedding_size, hidden_state_size, num_encoder_layers)
      #Defines Decoder Model
      self.decoder_model=ETPDecoder(num_phonetic_chars, embedding_size, hidden_state_size, num_decoder_layers, desired_output_length)
    else:
      self.encoder_model=ETPEncoderGRU(num_word_chars, embedding_size, hidden_state_size, num_encoder_layers)
      #Defines Decoder Model
      self.decoder_model=ETPDecoderGRU(num_phonetic_chars, embedding_size, hidden_state_size, num_decoder_layers, desired_output_length)
    self.num_phonetic_chars=num_phonetic_chars
    #Defines Maximum Output Length
    self.desired_output_length=desired_output_length
    #self.decoder=nn.Sequential()
  def forward(self, x, debug=False):
    #Encodes word
    
    encoder_result= self.encoder_model(x)[-1:];
  #  print("balls:\n{},\n {}:sllab".format(x.size(), encoder_result[1][0].size()))
    encoder_hidden=encoder_result[1][-1:]
    # print(encoder_hidden.size())
    if(debug):
       print(encoder_result);
    #Creates the initial list of zeroes for the output sequence
    batch_size=encoder_result[0].size()[0]
    y0=zeros(batch_size, self.desired_output_length)
  #  print(y0)
    #Sets the first item of the output sequence to the start token (SOS)
    y0[:,0]=start_token
  #  print(y0)
    #print(y0)
    #Initializes Empty list   
    rv=zeros(batch_size, self.desired_output_length, self.num_phonetic_chars)    
    #rv[0][1]=1
    rv[:, 0, 1]=1
    #print(rv)
    #rv[0][0]=0
  #  rv=[rv0]
    #Dummy variable for reporting last set of token probabilities in the case of an error
    pob=None
    #Defines the initial "y" values based on the encoder model
    y=(y0.long(), encoder_hidden);
    for i in range(self.desired_output_length-1):
      #Gets the decoder output for the y value
      # print("p00p")
      # print(y[0].size())
      # print(y[1].size())
      _y=self.decoder_model(y[0], y[1]);
      # print("prediction successful")
      if(debug):
         print("Decoder Prediction:")
         print(y0)
        #  print(_y[0].size())
        #  print(_y[1][0].size())
      
      #Gets the probability of each token as outputted by the decoder model
      pprob=_y[0]
      #Grabs the current sequence
      y0=y[0]
    # #  print("a")
    #  # print(self.decoder_feedforward(_y[0]))
    #   _y0=Variable(_y[0].data, requires_grad=True)
    #   lin=self.decoder_feedforward(_y0)
    #   pprob=self.decoder_activation(lin)
      #Defines the token probabilities as that for the most recent token
      pprob=pprob[:, self.desired_output_length-1]
      #print("X")
      #print(y0.size())
      #print(pprob.size())
      #Defines the next token in the sequence as the index of the greatest element of the phonetic probs 
      #print(pprob[:, 0])
      y0[:, i+1]=argmax(pprob, dim=1)
    #print(y0)
      #Reshapes by wrapping in it a big mama array
      rv[:, i+1]=pprob
      #y0=y0.view(tuple([1]+list(y0.size())))
   #   print(list(y0.size())) 
    #  print(reshape(y0, tuple([1]+list(y0.size()))))
     # print("b")
      #Sets y to the sequence and the last hidden state for the next
      y=(y0, encoder_hidden)
     # print("c")
      pob=pprob
    return (y, rv);

      
  def train_model(self, dataloader, loss_func, encoder_optimizer, decoder_optimizer, teacher_forcing_percent=0, num_datapoints=-1, repeat_per_batch=1):
    c=0;
    #Runs the train functions in normal nn.Module
    self.train();
    
    #Loops through every (x, y) pair in every batch in the Dataloader
    for batch, (x, y) in enumerate(dataloader):
      for i in range(repeat_per_batch):
        # print(x)
        #Gets the sequence prediction for x
        debug=(c%997==0)
        if(teacher_forcing_percent<random()):
 #         print(x.size())
          prediction=self(x, debug=False)
  #        print("yumyum")
          if(c%20==0):
            print(c)
            print(x)
            print(prediction)
            # print("a")
            # print(y)
          prediction=prediction[1]
          # print("c")
          # print(x)
          # print(y)
          # print(prediction.float())
      #    prediction=torch.stack(prediction)

        # print(prediction)
          prediction=Variable(prediction.data, requires_grad=True)
          batch_size=prediction.size()[0]
          #Gets the loss between the prediction and y
          target=zeros_like(prediction)
          print("target shape:"+str(target.size()))
          for i in range(self.desired_output_length):
            for j in range(batch_size):
              target[j, i, y[j][i]]=1
          print(target)
          loss=loss_func(prediction, target)
          
          if(c%5==0):
            print(loss)
            # print(prediction)
            # print(target)
        
        # print("d")
          #Zeroes the gradients of the optimizer
          encoder_optimizer.zero_grad(set_to_none=True)
          decoder_optimizer.zero_grad(set_to_none=True)
          #Connects loss to rest of network
          loss.backward()
          #Find dem gradients
          encoder_optimizer.step()
          decoder_optimizer.step()
        else:
        # print(encoder_result[1])
          encoder_result=self.encoder_model(x)
          loss=0
          batch_size=encoder_result[0].size()[0]
          prev_hidden=encoder_result[1][-1:]
         # print(y.size())
          for i in range(y.size()[1]-1):
              input=zeros(batch_size, y.size()[1]).long()
              input[:, :i+1]=y[:, :i+1].clone()
            # print(encoder_result[1])
              decoder_result=self.decoder_model(input.clone().view(batch_size, y.size()[1]), prev_hidden)
              prev_hidden=decoder_result[1][-1:]
              decoder_result=decoder_result[0][:, -1].clone()
              #print(decoder_result.size())
              target=zeros(batch_size, self.num_phonetic_chars)
              #print(target.size())
              for j in range(batch_size):
                target[j][y[j][i+1]]=1;
              new_loss=loss_func(decoder_result, target)
              # if(depend_on_variance):
              #   new_loss/=torch.sqrt(torch.var(decoder_result))
              loss+=new_loss
              if(c%10==0):
                print(c)
                print(new_loss)
              
              
            #Zeroes the gradients of the optimizer
          encoder_optimizer.zero_grad(set_to_none=True)
          decoder_optimizer.zero_grad(set_to_none=True)
          #Connects loss to rest of network
          loss.backward()
          #Find dem gradients
          encoder_optimizer.step()
          decoder_optimizer.step() 



        c+=1;
        if(c%20==0):
          torch.save(self, "/content/drive/MyDrive/ETPmodel_{}_{}.pt".format(c, str(datetime.now())))      
        
        if(c>num_datapoints and num_datapoints>0):
          return;




In [ ]:
print(str(indices))

[ 54328 125094   9969 ... 112087  42762  19219]


In [ ]:
f=open("indices_"+str(datetime.now())+".txt", "w")
f.write(str(indices))
f.close()

###Model Definition and Training

In [24]:
etpmodel=torch.load("/content/drive/MyDrive/ETPmodel_570_2022-12-24 04:47:12.714315.pt")

In [ ]:
etpmodel=ETPModel(len(letter_tokenizer.word_index)+1, len(phonetic_tokenizer.word_index)+1, 32, 8, 1, 1, use_gru=True).to(device)
#tisummary(etpmodel, (108124, 32), dtypes=[torch.long])


NameError: ignored

In [25]:
etpmodel=ETPModelV2(len(letter_tokenizer.word_index), len(phonetic_tokenizer.word_index), 64, 1024, 2, 2, desired_output_length=24, use_gru=True).to(device)

In [26]:
etpmodel.encoder_model.parameters=loaded_model.encoder_model.parameters
etpmodel.decoder_model.parameters=loaded_model.decoder_model.parameters

In [ ]:
etpmodel.train_model(etp_train_dataloader, nn.CrossEntropyLoss(ignore_index=0), optim.Adam(etpmodel.encoder_model.parameters(), lr=0.001), optim.Adam(etpmodel.decoder_model.parameters(), lr=0.001))

torch.Size([32, 24, 85])
Loss for data point 0: 35.32158660888672
,sos,s,w,eh1,s,ah0,n,s,eos,eos,eos,eos,eos,eos,eos,eos,eos,eos,eos,eos,eos,eos,eos,eos
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
Loss for data point 160: 26.857837677001953
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
Loss for data point 320: 29.250219345092773
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
Loss for data point 480: 40.425697326660156
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
Loss for data point 640: 28.941186904907227
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
torch.Size([32, 24, 85])
Loss for data point 800: 33.53500747680664
tor

In [ ]:
tisummary(etpmodel, (500, 32), dtypes=[torch.long])

In [ ]:
print(phonetic_tokenizer.word_index)

In [ ]:
#TensorBoard
summary_writer=SummaryWriter("/content/summarywriter")
summary_writer.add_graph(etpmodel, etp_train_dataset.tensors[0][:1])
summary_writer.close()

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir summarywriter

In [ ]:
etpmodel.train_model(etp_train_dataloader, nn.CrossEntropyLoss(), optim.Adam(etpmodel.encoder_model.parameters(), lr=0.01), optim.Adam(etpmodel.decoder_model.parameters(), lr=0.01), teacher_forcing_percent=0.5)

Streaming output truncated to the last 5000 lines.
140
tensor(0.3474, grad_fn=<DivBackward1>)
140
tensor(0.3474, grad_fn=<DivBackward1>)
140
tensor(0.3474, grad_fn=<DivBackward1>)
140
tensor(0.3474, grad_fn=<DivBackward1>)
140
tensor(0.3474, grad_fn=<DivBackward1>)
140
tensor(0.3474, grad_fn=<DivBackward1>)
140
tensor(0.3474, grad_fn=<DivBackward1>)
140
tensor(0.3474, grad_fn=<DivBackward1>)
140
tensor(0.3474, grad_fn=<DivBackward1>)
target shape:torch.Size([1, 32, 85])
tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]])
target shape:torch.Size([1, 32, 85])
tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
       

KeyboardInterrupt: ignored

In [ ]:
torch.save(etpmodel, "/content/drive/MyDrive/ETPmodel.pt")

'/content'

In [ ]:
word_index_inverted={v: k for k,v in phonetic_tokenizer.word_index.items()}
result=etpmodel(torch.tensor(letter_tokenizer.texts_to_sequences("'bout'")).long())[0][0][0]
for i in result:
  print(word_index_inverted[i.item()])

sos
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
iy0
aa0
aa1
aa1


In [ ]:
print(len(list(enumerate(etp_train_dataloader))))

108124


In [ ]:
for batch, (x, y) in enumerate(etp_train_dataloader):
  print(etpmodel(x))

In [ ]:

etpencodermodel=Sequential()
#tokenized_phonetics=tokenized_phonetics.reshape(tokenized_phonetics.shape[0], tokenized_phonetics.shape[1], 1).astype('float32')

print(tokenized_words[:1000])
etpmodel.add(Bidirectional(LSTM(word_length, return_sequences=True)))
etpmodel.add(Bidirectional(LSTM(2, return_sequences=True)))

etpmodel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['acc'])

In [ ]:
print()

28


In [ ]:
etpmodel.fit(tokenized_words, tokenized_phonetics, epochs=3, validation_split=0.1)

Epoch 1/3
3802/3802 [==============================] - 145s 36ms/step - loss: 217.1068 - acc: 0.0985 - val_loss: 228.2661 - val_acc: 0.1805
Epoch 2/3
3802/3802 [==============================] - 135s 36ms/step - loss: 223.2668 - acc: 0.1957 - val_loss: 246.7397 - val_acc: 0.1805
Epoch 3/3
3802/3802 [==============================] - 134s 35ms/step - loss: 246.0293 - acc: 0.1957 - val_loss: 271.9679 - val_acc: 0.1805


In [ ]:
word="balls"
tokenized_word_test=np.array(letter_tokenizer.texts_to_sequences([list(word)]))
print(tokenized_word_test)

[[17  2  9  9  5]]


In [ ]:
print(etpmodel(tokenized_word_test.reshape(tokenized_word_test.shape[0], tokenized_word_test.shape[1], 1)))

tf.Tensor(
[[1.17963425e-26 4.67159884e-26 2.56787429e-26 2.14984857e-26
  3.22911522e-26 1.49747339e-26 2.03534264e-26 2.97221404e-26
  2.19658689e-26 4.62203712e-26 1.21531270e-25 1.12930615e-24
  1.66428336e-23 9.15431353e-22 9.30941438e-20 1.69180114e-16
  5.07423747e-13 1.65561387e-09 6.24991969e-07 2.17812922e-04
  3.88885126e-03 8.19307044e-02 7.55403265e-02 9.99388546e-02
  1.32787436e-01 3.24861586e-01 1.51023388e-01 1.29810423e-01]], shape=(1, 28), dtype=float32)
